In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
import torch.utils.data.dataloader as Data
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset,DataLoader
import sys
from tqdm import tqdm

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
#Myscore function
def myScore(Target, Pred):
    tmp1 = 0
    tmp2 = 0
    for i in range(len(Target)):
        if Target[i] > Pred[i]:
            tmp1 = tmp1 + math.exp((-Pred[i] + Target[i]) / 13.0) - 1
        else:
            tmp2 = tmp2 + math.exp((Pred[i] - Target[i]) / 10.0) - 1
    tmp = tmp1 + tmp2
    return tmp

In [4]:
prediction_loss = []
# Load preprocessed data
X_train = sio.loadmat('./F001_window_size_trainX.mat')  # load sliding window preprocessed and Statistical features processed data (mean value and regression coefficient estimates feature)
X_train = X_train['train1X']

# X_train = X_train.reshape(len(X_train),42,14)
Y_train = sio.loadmat('./F001_window_size_trainY.mat')
Y_train = Y_train['train1Y']
Y_train = Y_train.transpose()

X_test = sio.loadmat('./F001_window_size_testX.mat')
X_test = X_test['test1X']
# X_test = X_test.reshape(len(X_test),42,14)
Y_test = sio.loadmat('./F001_window_size_testY.mat')
Y_test = Y_test['test1Y']
Y_test = Y_test.transpose()

In [5]:
print(X_train.shape)
print(Y_train.shape)

(16731, 40, 17)
(16731, 1)


In [6]:
X_train = Variable(torch.Tensor(X_train).float())
Y_train = Variable(torch.Tensor(Y_train).float())
X_test = Variable(torch.Tensor(X_test).float())
Y_test = Variable(torch.Tensor(Y_test).float())

In [7]:
print(X_test.shape)
print(Y_test.shape)

torch.Size([100, 40, 17])
torch.Size([100, 1])


In [8]:
#Hyperparameters
batch_size = 256
dim_val = 32
dim_attn = 32
dim_val_t = 32
dim_attn_t = 32
dim_val_s = 32
dim_attn_s = 32
n_heads = 4
n_decoder_layers = 1
n_encoder_layers = 2
max_rul = 125
lr = 0.001
epochs = 50
time_step = 40
dec_seq_len = 4
output_sequence_length = 1
input_size = 17

In [9]:
#Dataloader
train_dataset = TensorDataset(X_train,Y_train)
train_loader = Data.DataLoader(dataset=train_dataset,batch_size = batch_size,shuffle=True)
test_dataset = TensorDataset(X_test,Y_test)
test_loader = Data.DataLoader(dataset=test_dataset,batch_size = batch_size,shuffle=False)

In [12]:
# print(train_dataset[0:1])
# print('=' * 60)
# # 循环取数据
# for x_train, y_label in train_dataset:
#     print(x_train, y_label)

In [10]:
# Initialize model parameters
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# debug = True
debug = False
model = DAST(dim_val_s,dim_attn_s,dim_val_t,dim_attn_t,dim_val, dim_attn,time_step,input_size,dec_seq_len,output_sequence_length, n_decoder_layers, n_encoder_layers, n_heads, debug)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [14]:
# model._modules

In [11]:
print(next(model.parameters()).device)

cuda:0


In [16]:
# model.children()

In [12]:
#Training  and testing
loss_list = []
train_loss_list = []
test_loss_list = []
train_time = []
test_time = []
model_loss = 1000

In [13]:
for epoch in range(epochs):
    #training
    model.train()
    start1 = time.time()
    loop = tqdm(train_loader, leave=True)
    for i,(X, Y) in enumerate(loop):
        batch_X = X.to(device)
        batch_Y = Y.to(device)
        out = model(batch_X)
        # print(out[0])
        # print(out.next)
        loss = torch.sqrt(criterion(out*max_rul, batch_Y*max_rul))
        # print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    end1 = time.time()
    train_time.append(end1 - start1)
    loss_eopch = np.mean(np.array(loss_list))
    train_loss_list.append(loss_eopch)
    print('epoch = ',epoch,
            'train_loss = ',loss_eopch.item())

    #testing
    model.eval()
    prediction_list = []
    for j ,(batch_x,batch_y) in enumerate(test_loader):
        batch_X = batch_x.to(device)
        batch_Y = batch_y.to(device)
        start2= time.time()
        prediction = model(batch_X)
        end2 = time.time()
        test_time.append(end2 - start2)
        prediction[prediction<0] = 0
        prediction_list.append(prediction)

    out_batch_pre = torch.cat(prediction_list).detach().cpu().numpy()
    prediction_tensor = torch.from_numpy(out_batch_pre)
    test_loss = torch.sqrt(criterion(prediction_tensor*125, Y_test*125))
    test_loss_list.append(test_loss.item())
    Y_test_numpy = Y_test.detach().numpy()
    test_score = myScore(Y_test_numpy*125, out_batch_pre*125)
    print('test_loss = ', test_loss.item(),
            'test_score = ', test_score)
    

  0%|          | 0/66 [00:00<?, ?it/s]

100%|██████████| 66/66 [00:03<00:00, 20.91it/s]


epoch =  0 train_loss =  24.6486910906705
test_loss =  20.436994552612305 test_score =  1474.356894018513


100%|██████████| 66/66 [00:03<00:00, 20.92it/s]


epoch =  1 train_loss =  22.162899840961803
test_loss =  18.48623275756836 test_score =  1181.517507094448


100%|██████████| 66/66 [00:03<00:00, 20.58it/s]


epoch =  2 train_loss =  20.385028954708215
test_loss =  15.055702209472656 test_score =  416.6132002347805


  2%|▏         | 1/66 [00:00<00:05, 11.33it/s]


KeyboardInterrupt: 

In [69]:
print(type(prediction_list))

<class 'list'>
